In [20]:
"""
4개의 보드를 부모로 사용 후 적합도를 검사해 2개의 보드를 부모삼아 새로운 보드를 생성, 총 4개의 자식을 생성
적합도는 뭉쳐서 그려진 캔디 그룹의 수가 낮을수록 (움직임이 유효한 캔디가 많을수록) 높음.
"""

import pygame
from pygame.locals import *
import random

# 게임 창 세팅
width = 400
height = 400
scoreboard_height = 25
window_size = (width, height + scoreboard_height)

# 7종류의 캔디
candy_colors = ['blue', 'green', 'orange', 'pink', 'purple', 'red', 'teal', 'yellow']

# 그려질 캔디의 크기
candy_width = 40
candy_height = 40
candy_size = (candy_width, candy_height)

#실제 게임에서 쓰일 보드
board = []

class Candy:

    def __init__(self, row_num, col_num, color):

        # 캔디 위치 저장
        self.row_num = row_num
        self.col_num = col_num

        self.color = candy_colors[color] #보드에 적혀있는대로 캔디에 색깔 할당
        image_name = f'swirl_{self.color}.png'
        self.image = pygame.image.load(image_name)
        self.image = pygame.transform.smoothscale(self.image, candy_size)
        self.rect = self.image.get_rect()
        self.rect.left = col_num * candy_width
        self.rect.top = row_num * candy_height

# 3개 이상의 캔디로 뭉친 그룹 찾기
def find_matches(board, candy, matches, direction):

    matches.add(candy)

    # 위 체크
    if direction == "up" and candy.row_num > 0:
        neighbor = board[candy.row_num - 1][candy.col_num]
        if candy.color == neighbor.color and neighbor not in matches:
            matches.update(find_matches(board, neighbor, matches, direction))

    # 아래 체크
    if direction == "down" and candy.row_num < height / candy_height - 1:
        neighbor = board[candy.row_num + 1][candy.col_num]
        if candy.color == neighbor.color and neighbor not in matches:
            matches.update(find_matches(board, neighbor, matches, direction))

    # 왼쪽 체크
    if direction == "left"  and candy.col_num > 0:
        neighbor = board[candy.row_num][candy.col_num - 1]
        if candy.color == neighbor.color and neighbor not in matches:
            matches.update(find_matches(board, neighbor, matches, direction))

    # 오른쪽 체크
    if direction == "right" and candy.col_num < width / candy_width - 1:
        neighbor = board[candy.row_num][candy.col_num + 1]
        if candy.color == neighbor.color and neighbor not in matches:
            matches.update(find_matches(board, neighbor, matches, direction))

    return matches

# 캔디 그룹이 3개이상이면 그 그룹을, 아니면 빈 세트를 반환
def match_three(board, candy, direction):

    matches = find_matches(board, candy, set(), direction)
    if len(matches) >= 3:
        return matches
    else:
        return set()



boards = []
# boards: 보드 4개로 이루어진 삼중배열
# board[x][n][m]: (n, m)번째 있는 x번째 Candy Board의 값.

#총 4개의 보드를 생성할 것임
for i in range(4):

    # 이차원 리스트를 저장할 리스트 삽입
    boards.append([])

    for row_num in range(height // candy_height):

        # 이차원 리스트 삽입
        boards[i].append([])

        for col_num in range(width // candy_width):

            # 랜덤하게 이차원 리스트 값채우기
            candy = Candy(row_num, col_num, random.randint(0, 7))
            boards[i][row_num].append(candy)


print("Parent1-------------Parent2-------------Parent3-------------Parent4-------------")
for i in range(10):
    for k in range(4):
        for j in range(10):
            print(boards[k][i][j].color[0], end = " ")
        print(" ", end = "")
    print("\n", end = "")

#유전자 알고리즘 보드 생성 함

#변이
def Mutation(board):
    for row in board:
        for candy in row:
            if(random.randint(1, 100) == 1):
                candy.color = candy_colors[random.randint(0, 7)]
                image_name = f'swirl_{candy.color}.png'
                candy.image = pygame.image.load(image_name)
                candy.image = pygame.transform.smoothscale(candy.image, candy_size)

    print("\nchild--------")

    for i in range(10):
        for j in range(10):
            print(board[i][j].color[0], end = " ")
        print("")
    return board

#교차
def crossover(parent):
    row = random.randint(0, 10)
    col = random.randint(0, 10)
    print("row: ", row, "col: ", col)
    child = [[] for i in range(10)]

    for i in range(10):
        for j in range(10):
            print(parent[0][i][j].color[0], end = " ")
        print("")
    print("")
    for i in range(10):
        for j in range(10):
            print(parent[1][i][j].color[0], end = " ")
        print("")

    for i in range(row):
        for j in range(col):
            child[i].append(parent[0][i][j])
        for j in range(10 - col):
            child[i].append(parent[1][i][j + col])

    for i in range(10 - row):
        for j in range(col):
            child[i + row].append(parent[1][i + row][j])
        for j in range(10 - col):
            child[i + row].append(parent[0][i + row][j + col])

    return Mutation(child)

#룰렛 휠
def wheel(boards, fitness):
    sum = 0
    parent = []
    for i in fitness:
        sum += i
    print("")
    for i in range(2):
        rand = random.randint(1, sum)
        print("rand: ", rand)
        for j in range(4):
            if(rand <= fitness[j]):
                print(j+1)
                parent.append(boards[j])
                break
            else:
                rand -= fitness[j]
    return crossover(parent)


#보드의 적합도 검사
def selection(board):
    unfitness = 0
    for row in board:
        for candy in row:
            tmp = match_three(board, candy, "up")
            if(len(tmp) >= 3):
                unfitness += len(tmp)
            tmp = match_three(board, candy, "down")
            if(len(tmp) >= 3):
                unfitness += len(tmp)
            tmp = match_three(board, candy, "left")
            if(len(tmp) >= 3):
                unfitness += len(tmp)
            tmp = match_three(board, candy, "right")
            if(len(tmp) >= 3):
                unfitness += len(tmp)
    return unfitness

#유전자 알고리즘으로 보드 생성
gene = 0
while True:
    print("\n\ngeneration: ", gene)
    selected_board = 4
    fit = []
    print("Fitness: ", end = "")

    for i in range(4):
        fit.append(100 - selection(boards[i]))

    print(fit)

    for i in range(4):
        if(fit[i] == 100):
            selected_board = i

    if(selected_board < 4):
        board = boards[selected_board]
        break

    for board in boards:
        board = wheel(boards, fit)

    gene += 1

Parent1-------------Parent2-------------Parent3-------------Parent4-------------
p g o p g g g r y r  o y b y b b y y g p  y b t y p p b p p y  p b t p o b p y y b  
b y y y p r t b r r  g t t p b y t o r o  b b y b r r g p p y  r p y r o p p t o o  
r p r t p o t g g r  p r g t p g t y b p  t b g t p b r b p o  r p t p b y y b b g  
g b t y r b b b b p  b p b o g p y b r p  b p o y r r p g g p  b b y y p y y p p y  
p p p b t r p r y p  t g b o g y t o b p  g t t p g p y b t t  p g p g t p o b o t  
t t g r o t g p o p  b p g p y r r t g y  o p o p y r g p r y  o y r g g r t r b g  
y r b r g o p y p t  g p r p p g p o y o  b p o p p r o p p p  p p b y y y t o p r  
y g g r g g y o p y  p y b g r r t p r y  t y t o o g r r y g  p p g p p y t y g b  
t o p g b y p p r b  t r b p g t b r p y  t b t b b b p p p o  p p p t p g t t p r  
g b b b b r p g y r  p r t t t o y p y g  p p g p o o o p p p  p t p p g b b y p g  


generation:  0
Fitness: [48, 94, 76, 74]

rand:  110
2
rand:  203
3